


___________________
# **New Approach**
__________________

In [3]:
!pip install dash
!pip install pyngrok
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
from dash.dependencies import Input, Output
from pyngrok import ngrok

# Load data from CSV file
df = pd.read_csv('/content/company_data.csv')

# Initialize Dash app
app = dash.Dash(__name__)

# Light mode color scheme
colors = {
    'background': '#F5F5F5',
    'text': '#212121',
    'accent': '#0277BD',  # Blue accent
    'highlight': '#FFC107',  # Yellow accent
}

# Layout
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hi-Tech Manufacturing Dashboard',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='A dashboard for monitoring hi-tech manufacturing metrics.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='waste-recycling',
        figure={
            'data': [
                go.Bar(
                    x=df['year'],
                    y=df['waste_generated'],
                    name='Total Waste Generated',
                    marker=dict(color=colors['accent'])
                ),
                go.Bar(
                    x=df['year'],
                    y=df['waste_recycled'],
                    name='Waste Recycled',
                    marker=dict(color=colors['highlight'])
                )
            ],
            'layout': go.Layout(
                title='Waste Generation and Recycling',
                xaxis={'title': 'Year'},
                yaxis={'title': 'Waste (tons)'},
                barmode='stack',
                plot_bgcolor=colors['background'],
                paper_bgcolor=colors['background'],
                font={'color': colors['text']}
            )
        }
    ),
     dcc.Graph(
        id='waste-pie-chart',
        figure={
            'data': [
                go.Pie(
                    labels=['Waste Generated', 'Waste Recycled'],
                    values=[df['waste_generated'].sum(), df['waste_recycled'].sum()],
                    marker=dict(colors=[colors['accent'], colors['highlight']])
                )
            ],
            'layout': go.Layout(
                title='Waste Generation vs Recycling (Total)',
                plot_bgcolor=colors['background'],
                paper_bgcolor=colors['background'],
                font={'color': colors['text']}
            )
        }
    ),

    dcc.Graph(
        id='energy-consumption',
        figure={
            'data': [
                go.Scatter(
                    x=df['year'],
                    y=df['energy_consumption'],
                    mode='lines+markers',
                    marker=dict(color=colors['accent'])
                )
            ],
            'layout': go.Layout(
                title='Energy Consumption Over Years',
                xaxis={'title': 'Year'},
                yaxis={'title': 'Energy Consumption'},
                plot_bgcolor=colors['background'],
                paper_bgcolor=colors['background'],
                font={'color': colors['text']}
            )
        }
    ),

    html.Div(id='detailed-view', style={'color': colors['text']})
])

# Callback to update detailed view
@app.callback(
    Output('detailed-view', 'children'),
    [Input('waste-recycling', 'hoverData')]
)
def update_detailed_view(hoverData):
    if hoverData is not None:
        year = hoverData['points'][0]['x']
        total_waste_generated = df[df['year'] == year]['waste_generated'].values[0]
        waste_recycled = df[df['year'] == year]['waste_recycled'].values[0]
        return f'Detailed View: Year {year}, Total Waste Generated: {total_waste_generated}, Waste Recycled: {waste_recycled}'

if __name__ == '__main__':
    app.run_server(port=4050)
    ngrok_url = ngrok.connect(4050)
    print(f'Open the Dash app at: {ngrok_url}')


<IPython.core.display.Javascript object>

ERROR:pyngrok.process.ngrok:t=2024-03-24T17:19:06+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-03-24T17:19:06+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-03-24T17:19:06+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

________________
# **Adding LCA**
----------------
it is a quick and intuitive way for designers and engineers to understand, analyse and compare environmental impacts of products and particular design decisions

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2024-03-21 10:18:25--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  --.-KB/s    in 0.06s   

2024-03-21 10:18:25 (231 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [4]:
!pip install pyngrok
!pip install dash
from dash import dcc, html, Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import subprocess
from google.colab.output import eval_js

# Import Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Create Dash app
app = dash.Dash(__name__)

# Define layout for LCA calculator
app.layout = html.Div([
    html.H1('Life Cycle Assessment (LCA) Calculator'),
    html.Div([
        html.Label('Material Composition (kg):'),
        dcc.Input(id='material-input', type='number', placeholder='Material Used (kg)')
    ]),
    html.Div([
        html.Label('Energy Consumption (kWh):'),
        dcc.Input(id='energy-input', type='number', placeholder='Energy Consumption (kWh)')
    ]),
    html.Div([
        html.Label('Transportation Distance (km):'),
        dcc.Input(id='transportation-input', type='number', placeholder='Transportation Distance (km)')
    ]),
    # Add more input fields for additional parameters
    html.Div(id='lca-results'),
    html.Div(id='charts')
])

# Define callback to perform LCA calculation
@app.callback(
    Output('lca-results', 'children'),
    [Input('material-input', 'value'),
     Input('energy-input', 'value'),
     Input('transportation-input', 'value')]
)
def calculate_lca(material, energy, transportation):
    if all([material, energy, transportation]):
        # Perform simplified LCA calculation (for demonstration purposes)
        carbon_footprint = material * 0.5 + energy * 0.2 + transportation * 0.1
        water_usage = material * 0.3 + energy * 0.1 + transportation * 0.05
        # Display LCA results
        return html.Div([
            html.H3('Life Cycle Assessment (LCA) Results:'),
            html.P(f'Carbon Footprint: {carbon_footprint:.2f} kg CO2 eq.'),
            html.P(f'Water Usage: {water_usage:.2f} liters')
        ])
    else:
        raise PreventUpdate

# Define callback to generate charts
@app.callback(
    Output('charts', 'children'),
    [Input('material-input', 'value'),
     Input('energy-input', 'value'),
     Input('transportation-input', 'value')]
)
def generate_charts(material, energy, transportation):
    if all([material, energy, transportation]):
        # Generate donut charts
        fig_carbon = px.pie(names=['Material', 'Energy', 'Transportation'],
                            values=[material * 0.5, energy * 0.2, transportation * 0.1],
                            title='Carbon Footprint Breakdown')
        fig_water = px.pie(names=['Material', 'Energy', 'Transportation'],
                           values=[material * 0.3, energy * 0.1, transportation * 0.05],
                           title='Water Usage Breakdown')

        return [
            dcc.Graph(figure=fig_carbon),
            dcc.Graph(figure=fig_water)
        ]
    else:
        raise PreventUpdate

# Run the Dash app
if __name__ == '__main__':
    # Run the app
    app.run_server(debug=True, port=4050)
    # Use ngrok to create a public URL for accessing the app
    subprocess.Popen(['ngrok', 'http', '4050'])
    ngrok_url = eval_js("google.colab.kernel.proxyPort(4050)")
    print(f'Open the Dash app at: {ngrok_url}')


<IPython.core.display.Javascript object>

Open the Dash app at: https://9gmaspy3r09-496ff2e9c6d22116-4050-colab.googleusercontent.com/
